In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão.

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [11]:
# Baixa a base de dados.
housing = fetch_california_housing()

# Divide o conjunto total de exemplos em conjuntos de treinamento e teste.
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# Divide o conjunto de treinamento em conjuntos de treinamento (menor) e validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Aplica padronização às matrizes de atributos.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Salvando os pesos de modelos criados a partir de subclasses da classe `Model`.

+ O método `save()` e a função `load_model` funcionam ao usar as APIs sequencial e funcional, **mas, infelizmente, não ao usar subclasses da classe `Model`.**

+ Porém, podemos usar os métodos `save_weights()` e `load_weights()` para pelo menos salvar e restaurar os parâmetros do modelo.

+ Entretanto, precisaremos salvar e restaurar todo o resto nós mesmos.

In [12]:
class Regressor(keras.models.Model):

    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # lida com parâmetros padrão como, por exemplo, name.
        self.hidden = keras.layers.Dense(units, activation=activation, name="input")
        self.out = keras.layers.Dense(1)

    def call(self, inputs):
        hidden = self.hidden(inputs)
        out = self.out(hidden)
        return out

In [13]:
model = Regressor()

In [14]:
model.compile(
    loss="mse",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3)
)

In [15]:
# Treinando o modelo.
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    validation_data=(X_valid, y_valid)
                   )

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.4101 - val_loss: 0.8490
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8218 - val_loss: 0.7208
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7360 - val_loss: 0.6585
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6853 - val_loss: 0.6130
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6455 - val_loss: 0.5771
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6130 - val_loss: 0.5482
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.5860 - val_loss: 0.5242
Epoch 8/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.5634 - val_loss: 0.5043
Epoch 9/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.5442 - val_loss: 0.4876
Epoch 10/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.5277 - val_loss: 0.4740
Epoch 11/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.5139 - val_loss: 0.4629
Epoch 12/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

In [18]:
X_new = X_test[:3]

y_pred = model.predict(X_new)

for i in range(len(y_pred)):
    print('Actual: %1.3f - Predicted: %1.3f' % (y_test[i], y_pred[i,0]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Actual: 0.477 - Predicted: 0.483
Actual: 0.458 - Predicted: 1.642
Actual: 5.000 - Predicted: 3.264


In [19]:
model.save_weights("./my_keras_weights.weights.h5")

### Importante

+ Para carregar os pesos, **precisamos antes ter um modelo instanciado e compilado**.

In [20]:
loaded_model = Regressor()

loaded_model.compile(
    loss="mse",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3)
)

loaded_model.load_weights("./my_keras_weights.weights.h5")

Depois dos pesos terem sido carregados, podemos fazer predições normalmente.

In [21]:
y_pred = loaded_model.predict(X_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


In [22]:
for i in range(len(y_pred)):
    print('Actual: %1.3f - Predicted: %1.3f' % (y_test[i], y_pred[i,0]))

Actual: 0.477 - Predicted: 0.143
Actual: 0.458 - Predicted: 0.119
Actual: 5.000 - Predicted: -1.284
